In [1]:
# LIBRARIES
import numpy as np

import sys
sys.path.append('lib/')
from pca import *

In [2]:
def this_practical_qr_algorithm(matrix,k_max,zero_cutoff):
    m = matrix.shape[0]
    A = np.ones((k_max,m,m),dtype=float)
    A[0] = hessenberg(matrix)
    shift = np.ones(k_max)
    eig=[]
    should_return=False
    # print(matrix.round(5))
    for k in range(1,k_max):
        # print('k:',k)
        shift[k] = A[k-1,m-1,m-1]
        if m==1: 
            # print('last:',shift[k])
            # eig=eig+[shift[k].round(6)]
            eig=_combineEigenvalues(eig,[shift[k].round(6)])
            return((list(set(eig))))
        Q,R = householder(A[k-1,:,:]-shift[k]*np.identity(m))
        A[k,:,:] = np.matmul(R,Q)+shift[k]*np.identity(m)
        # print(A[k,:,:].round(5))
        for j in range(m-1):
            # print('  j:',j,'val:',A[k,j,j+1])
            if np.abs(A[k,j,j+1]) < zero_cutoff:
                should_return=True
                # print('shift:',shift[k],',',A[k,j,j+1])
                A[k,j,j+1]=0
                A[k,j+1,j]=0
                # print('shift:',shift[k],',',A[k,j,j+1])
                # break
                A_1 = A[k,:j+1,:j+1]
                A_2 = A[k,j+1:,j+1:]
                # eig=eig+[shift[k].round(6)]
                eig=_combineEigenvalues(eig,[shift[k].round(6)])
                if A_1.shape[0]>0: 
                    e_temp = this_practical_qr_algorithm(A_1,k_max,zero_cutoff)
                    # print('eig:',eig,'etemp:',e_temp)
                    # print(e_temp)
                    # if abs(e_temp[0])>0:eig=eig+e_temp
                    # if e_temp!=None:eig=eig+e_temp
                    if e_temp!=None:eig=_combineEigenvalues(eig,e_temp)
                if A_2.shape[0]>0: 
                    e_temp = this_practical_qr_algorithm(A_2,k_max,zero_cutoff)
                    # print('eig:',eig,'etemp:',e_temp)
                    # print(e_temp)
                    # if abs(e_temp[0])>0:eig=eig+e_temp
                    # if e_temp!=None:eig=eig+e_temp
                    if e_temp!=None:eig=_combineEigenvalues(eig,e_temp)
        # if should_return: return((list(set(eig))))

    # print('not found')  
    # return(shift[k])
    # print(eig)
    # eig = [ round(e, 6) % e for e in eig ] # round each value in eig list
    # print('hi')
    return((list(set(eig))))

def _combineEigenvalues(old,new):
    # assuming new values will be more accurate
    # print('combining')
    # print(old)
    # print(new)
    # if old != []:
    for o in old:
        should_add = True
        for n in new:
            p = abs(n*.1)
            if o>n-p and o<n+p: should_add = False
            # print('o:',o,'n:',n,'p:',p,'should_add:',should_add)
        if should_add: new = [o] + new
    
    # print('returning:',new)
    return(new)

In [3]:
A = np.array(
    [[1,2,3,4],
    [2,1,2,2],
    [3,2,1,3],
    [4,2,3,1]]
)

# A[0,0:A.shape[0]]
print(this_practical_qr_algorithm(A,20,1e-8))
print(practical_qr_algorithm(A,20,1e-8))
print(np.linalg.eig(A)[0])

# j=2
# print(A[:j,:j])
# print(A[j:,j:])

[-0.446587, 9.158122, -3.0, -1.711536]
[-3.00000002, 9.15812232, -0.44658652, -1.71153587]
[ 9.15812238 -3.         -0.4465865  -1.71153588]


In [4]:
j=2
print(A)
A_1 = A[:j+1,:j+1]
A_2 = A[j+1:,j+1:]

print(A_1)
print(A_2)

[[1 2 3 4]
 [2 1 2 2]
 [3 2 1 3]
 [4 2 3 1]]
[[1 2 3]
 [2 1 2]
 [3 2 1]]
[[1]]


In [5]:
# TRY FOR SOME S
# randomly generate some data matrix X
n = 5 # number of data points
p = 3 # number of features

X = np.random.randint(10,size=(n,p))

X_mean_array = np.mean(X,axis=0)
ones = np.ones(n)
X_mean = np.outer(ones,X_mean_array)
X_diff = X-X_mean
S = np.matmul(X_diff,X_diff.T)/n


print(this_practical_qr_algorithm(S,5,1e-5))
print(practical_qr_algorithm(S,5,1e-5))
print(np.linalg.eig(S)[0])

[0.0, 17.43253, 0.725884, 6.801586]
[6.80158564, 17.4325305, 0.72588375, 0.0]
[ 1.74325307e+01  6.80158555e+00  7.25883744e-01  5.46567261e-16
 -4.78359982e-16]


6.80158564
